In [68]:
import datetime
import json
import os
import time

from tqdm import tqdm
from tweepy import TweepyException

DATA_PATH='data'
from twitter_utils import get_client, get_profiles


In [11]:
query = "((((abuso OR violenza OR sessuale OR vittima) (minore OR minorenne OR minori OR minorenni)) OR (pedofilo OR pedofilia OR pedofili)) (sacerdote OR don OR prete OR papa OR vescovo OR cardinale OR prelato OR parroco OR sacerdoti OR preti OR vescovi OR cardinali OR prelati OR parroci OR chiesa OR cei OR episcopale OR episcopali OR clero OR cattolico OR cattolica OR cattolici OR parrocchia OR parrocchie OR parrocchiale OR parrocchiali OR ecclesiastico OR ecclesiastica OR ecclesiastici)) OR (italychurchtoo OR #italychurchtoo OR @italychurchtoo) OR (from:RLABUSO OR from:italychurchtoo)"

# query = "((((abuso OR violenza OR sessuale OR vittima) (minore OR minorenne OR minori OR minorenni)) OR (pedofilo OR pedofilia OR pedofili)))"



In [12]:
# ( AND (don OR prete OR papa OR vescovo OR cardinale OR prelato OR parroco OR preti OR  OR vescovi OR cardinali OR prelati OR parroci OR chiesa OR cei OR episcopale OR clero OR cattolico OR cattolica OR cattolici OR parrocchia OR parrocchie)) OR (italychurchtoo OR #italychurchtoo OR @italychurchtoo) OR from:RLABUSO OR from:italychurchtoo

In [69]:
client = get_client()


In [70]:
out_path = os.path.join(DATA_PATH, f'itlaychurchtoo_nolang.jsonl')
errors = list()

In [72]:
start_date = datetime.datetime(2010, 1, 1, 0, 0, 0)
end_date = None


In [20]:
if os.path_exists(out_path):
    with open(out_path) as f:
        for l in f:
            pass
    end_date = json.loads(l)['created_at']

In [30]:
with open(out_path, 'a+') as f:
    done = False
    next_token = None
    while not done:
        try:
            res = client.search_all_tweets(query=query, next_token=next_token, max_results=500, start_time=start_date, end_time=end_date,\
                                           place_fields = ['id','full_name', 'country', 'geo', 'name', 'place_type'],
                                         tweet_fields = ['id', 'created_at', 'geo', 'public_metrics', 'text', 'edit_history_tweet_ids', 'attachments', 'author_id',  'conversation_id', 'entities', 'in_reply_to_user_id', 'lang', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'withheld'], # missing edit controls and  other fields that require user auth, 'context_annotations' requires <100 results
                        media_fields=['url', 'alt_text','public_metrics', 'type', 'media_key', 'duration_ms'],  \
                        poll_fields=['id', 'options', 'duration_minutes', 'end_datetime', 'voting_status'],
                                           since_id=None, sort_order=None, \
                        until_id=None, user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id',
               'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'])
            for tweet in res.data:
                f.write(json.dumps(tweet.data, sort_keys=True)+'\n')
            next_token = res.meta.get('next_token', None)
            if not next_token:
                done=True

        except TweepyException as e:
            print(e)
            errors.extend(query)
            time.sleep(60)
            # done=True

In [31]:
cnt = 0
with open(out_path) as f:
    for l in f:
        cnt+=1
cnt

737787

In [34]:
authors = set()
with open(out_path) as f:
    for l in f:
        authors.add(json.loads(l)['author_id'])

In [35]:
len(authors)

368914

In [37]:

out_path = os.path.join(DATA_PATH, 'italychurchtoo_rehydrated.jsonl')
err_path = os.path.join(DATA_PATH, 'italychurchtoo_missing.jsonl')

author_errors = get_profiles(ids=list(authors), client=client, out_path=out_path, error_path=err_path)

batch: 100%|██████████| 3690/3690 [3:00:49<00:00,  2.94s/it]   


In [73]:
out_path = os.path.join(DATA_PATH, f'itlaychurchtoo_nolang.jsonl')
conversations = set()
with open(out_path) as f:
    for l in f:
        tweet = json.loads(l)
        # should filter out tweets that are original (or retweets/quote retweets) but have no replies. would retweets create new conversation_ids? yes, so them too.
        if (('referenced_tweets' not in tweet) or (tweet['referenced_tweets'][0]['type']!='reply')) and (tweet["public_metrics"]["reply_count"]==0):
            continue
        conversations.add(tweet['conversation_id'])

In [74]:
len(conversations)

43313

In [75]:
queries = list()

while(len(conversations)):
    query = ''
    while len(query)<=1024:
        if not len(conversations):
            break
        next_id= conversations.pop()
        next_id_str=f"conversation_id:{next_id} OR "
        if (len(query)+len(next_id_str)>1024):
            query = query[:-len(" OR ")]
            conversations.add(next_id)
            break
        else:
            query+=next_id_str
    queries.append(query)

In [76]:
len(queries)

1666

In [77]:
out_path = os.path.join(DATA_PATH, f'italychurchtoo_nolang_convo.jsonl')
errors = list()
with open(out_path, 'a+') as f:
    for query in tqdm(queries, 'queries'):
        done = False
        next_token = None
        while not done:
            try:
                res = client.search_all_tweets(query=query, next_token=next_token, max_results=500, start_time=start_date, end_time=None,\
                                               place_fields = ['id','full_name', 'country', 'geo', 'name', 'place_type'],
                                             tweet_fields = ['id', 'created_at', 'geo', 'public_metrics', 'text', 'edit_history_tweet_ids', 'attachments', 'author_id',  'conversation_id', 'entities', 'in_reply_to_user_id', 'lang', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'withheld'], # missing edit controls and  other fields that require user auth, 'context_annotations' requires <100 results
                            media_fields=['url', 'alt_text','public_metrics', 'type', 'media_key', 'duration_ms'],  \
                            poll_fields=['id', 'options', 'duration_minutes', 'end_datetime', 'voting_status'],
                                               since_id=None, sort_order=None, \
                            until_id=None, user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id',
                   'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'])
                for tweet in res.data:
                    f.write(json.dumps(tweet.data, sort_keys=True)+'\n')
                next_token = res.meta.get('next_token', None)
                if not next_token:
                    done=True

            except TweepyException as e:
                print(e)
                errors.append(query)
                time.sleep(60)
                done=True

queries:  15%|█▍        | 245/1666 [5:09:42<31:38:23, 80.16s/it] 

503 Service Unavailable
Service Unavailable


queries:  27%|██▋       | 446/1666 [12:13:16<14:34:52, 43.03s/it]  

503 Service Unavailable
Service Unavailable


queries:  32%|███▏      | 528/1666 [15:41:19<21:45:27, 68.83s/it]  

...

503 Service Unavailable
Service Unavailable


queries:  73%|███████▎  | 1212/1666 [39:12:35<23:39:16, 187.57s/it]

503 Service Unavailable
Service Unavailable


queries:  75%|███████▌  | 1255/1666 [40:43:21<7:41:27, 67.37s/it]  Rate limit exceeded. Sleeping for 252 seconds.
Rate limit exceeded. Sleeping for 238 seconds.
queries:  78%|███████▊  | 1306/1666 [42:15:00<59:10,  9.86s/it]    Rate limit exceeded. Sleeping for 734 seconds.
Rate limit exceeded. Sleeping for 901 seconds.
Rate limit exceeded. Sleeping for 901 seconds.
Rate limit exceeded. Sleeping for 901 seconds.
Rate limit exceeded. Sleeping for 901 seconds.
queries:  78%|███████▊  | 1306/1666 [43:27:52<11:58:51, 119.81s/it]


KeyboardInterrupt: 

In [79]:
len(queries)

1666

In [88]:
remaining = queries[1304:]+["conversation_id:837194261326147584 OR conversation_id:1040019666771603472 OR conversation_id:260513316135378946 OR conversation_id:1236873220063551488 OR conversation_id:1314599168120414209 OR conversation_id:1086055188862062592 OR conversation_id:1033467210226978817 OR conversation_id:844327187486593024 OR conversation_id:385163694533255168 OR conversation_id:383601963549868032 OR conversation_id:209263740821061633 OR conversation_id:1486399242885373962 OR conversation_id:515432217255104512 OR conversation_id:1279391502041481216 OR conversation_id:1609808498677653506 OR conversation_id:1063063349116592128 OR conversation_id:1452979688322785290 OR conversation_id:1099070912752730112 OR conversation_id:1067480544978493440 OR conversation_id:1409057350103805952 OR conversation_id:994196327826362368 OR conversation_id:1266592887107715073 OR conversation_id:1268627780540874752 OR conversation_id:1606488571606605824 OR conversation_id:1603590403667955713 OR conversation_id:1542892328012201989",
"conversation_id:1038493214771167233 OR conversation_id:1586360919595966466 OR conversation_id:1166258903929237504 OR conversation_id:705417127528951809 OR conversation_id:955470119055978496 OR conversation_id:1134458316875292673 OR conversation_id:881853108694994944 OR conversation_id:1274070209255653377 OR conversation_id:1500661157585047554 OR conversation_id:1100466157834907651 OR conversation_id:310872385702223872 OR conversation_id:191835414917689344 OR conversation_id:982257389712719883 OR conversation_id:1136250500326875139 OR conversation_id:651644548754612224 OR conversation_id:557502481761320962 OR conversation_id:1610045988693020677 OR conversation_id:1556414077441818624 OR conversation_id:148129100412887041 OR conversation_id:1540715089778688000 OR conversation_id:1326247136577531904 OR conversation_id:652099988730855424 OR conversation_id:964872633115381760 OR conversation_id:831840602815885313 OR conversation_id:251082925653770241 OR conversation_id:1465458347285651461",
"conversation_id:1278965541705846785 OR conversation_id:1560318731804344329 OR conversation_id:1611659791532978176 OR conversation_id:1479132545899974662 OR conversation_id:1030134624746438656 OR conversation_id:648226438580187136 OR conversation_id:1266130776929730560 OR conversation_id:307157456306442240 OR conversation_id:900276844708188160 OR conversation_id:1235782768556224512 OR conversation_id:1591483304074936320 OR conversation_id:1482864292261203968 OR conversation_id:212645159400247296 OR conversation_id:1024976198231678976 OR conversation_id:1564377876287438850 OR conversation_id:1152919825611669509 OR conversation_id:1128971683430375424 OR conversation_id:654429881372123136 OR conversation_id:1235681213635063816 OR conversation_id:1134085821580161026 OR conversation_id:999405008595636224 OR conversation_id:839386285638889472 OR conversation_id:1034007220135096320 OR conversation_id:781998832796200960 OR conversation_id:1019423872385454080 OR conversation_id:589390374143012864",
"conversation_id:1415504184804261888 OR conversation_id:1030747586595106816 OR conversation_id:663248916092203010 OR conversation_id:635186714378526722 OR conversation_id:1609202384177070080 OR conversation_id:958688034572169218 OR conversation_id:1494051761694326787 OR conversation_id:1593585507564339203 OR conversation_id:488701944698253313 OR conversation_id:1142062643756318720 OR conversation_id:483511118733012992 OR conversation_id:1594143281050537986 OR conversation_id:1614608414440669184 OR conversation_id:983530482875936768 OR conversation_id:1609595624306970626 OR conversation_id:731893570324578304 OR conversation_id:1021773926337273857 OR conversation_id:978334518305320960 OR conversation_id:1144312015957180418 OR conversation_id:468739499061968896 OR conversation_id:1030461370645585920 OR conversation_id:1042014027994554368 OR conversation_id:1425916925863993356 OR conversation_id:849595040091574272 OR conversation_id:586431287776296960 OR conversation_id:1043584146608726016",
"conversation_id:1488348857910104066 OR conversation_id:1076164981413040129 OR conversation_id:294960847372029952 OR conversation_id:1582798435383619586 OR conversation_id:557205913497772032 OR conversation_id:682235360726790144 OR conversation_id:1011292402462724096 OR conversation_id:905478360909963265 OR conversation_id:1108002680222433280 OR conversation_id:622398151576694786 OR conversation_id:1592428338080993280 OR conversation_id:807583978320527360 OR conversation_id:603480427622457344 OR conversation_id:1115288629755240449 OR conversation_id:1220462549197606914 OR conversation_id:688029834606473217 OR conversation_id:1600865103645093889 OR conversation_id:311226979905060864 OR conversation_id:1371006926876725249 OR conversation_id:439602814495764480 OR conversation_id:311277356096581632 OR conversation_id:1216826778939011083 OR conversation_id:113636391598034945 OR conversation_id:390219634290724864 OR conversation_id:846117735331450880 OR conversation_id:1322154969391374336",
"conversation_id:1372517340055793666 OR conversation_id:313242004664492033 OR conversation_id:1164584267831271424 OR conversation_id:1029448433453686784 OR conversation_id:350811939372404737 OR conversation_id:1034135889566412800 OR conversation_id:1003057506329952256 OR conversation_id:1068519826224701444 OR conversation_id:1054857503622791168 OR conversation_id:1514349312959991809 OR conversation_id:1331636723181293569 OR conversation_id:514913797669388288 OR conversation_id:1036182399330205696 OR conversation_id:879403161810808833 OR conversation_id:431332532093984768 OR conversation_id:726076817098985472 OR conversation_id:639406294005350400 OR conversation_id:1052673776486367239 OR conversation_id:966133171782930432 OR conversation_id:1175295765482278912 OR conversation_id:684107516595335168 OR conversation_id:1611274677992960002 OR conversation_id:1094912124063793152 OR conversation_id:512324496414740480 OR conversation_id:1272705735776968704 OR conversation_id:611238876783448064",
"conversation_id:1072891329607278593 OR conversation_id:1224195006845472768 OR conversation_id:1481222718649425920 OR conversation_id:1023653553645207552 OR conversation_id:822803316094726145 OR conversation_id:1131105439289094144 OR conversation_id:1226117273116663810 OR conversation_id:1202753930352058368 OR conversation_id:646741718902177792 OR conversation_id:983326646756311040 OR conversation_id:210023715331776512 OR conversation_id:1530927124051378177 OR conversation_id:166608432219164673 OR conversation_id:458944649609375744 OR conversation_id:445297172431794177 OR conversation_id:912812575661977600 OR conversation_id:1199666682731479043 OR conversation_id:1444602186177056775 OR conversation_id:307162268322394112 OR conversation_id:226416720133369856 OR conversation_id:1339607960524353546 OR conversation_id:1570615484223225857 OR conversation_id:1537042585612697601 OR conversation_id:1040187538177236992 OR conversation_id:963038404689711105 OR conversation_id:208647860718280705"]

In [89]:
len(remaining)

369

In [91]:
import tweepy

client = tweepy.Client(bearer_token="XXX",
                         wait_on_rate_limit=True, )

In [92]:
out_path = os.path.join(DATA_PATH, f'italychurchtoo_nolang_convo.jsonl')
errors = list()
with open(out_path, 'a+') as f:
    for query in tqdm(remaining, 'queries'):
        done = False
        next_token = None
        while not done:
            try:
                res = client.search_all_tweets(query=query, next_token=next_token, max_results=500, start_time=start_date, end_time=None,\
                                               place_fields = ['id','full_name', 'country', 'geo', 'name', 'place_type'],
                                             tweet_fields = ['id', 'created_at', 'geo', 'public_metrics', 'text', 'edit_history_tweet_ids', 'attachments', 'author_id',  'conversation_id', 'entities', 'in_reply_to_user_id', 'lang', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'withheld'], # missing edit controls and  other fields that require user auth, 'context_annotations' requires <100 results
                            media_fields=['url', 'alt_text','public_metrics', 'type', 'media_key', 'duration_ms'],  \
                            poll_fields=['id', 'options', 'duration_minutes', 'end_datetime', 'voting_status'],
                                               since_id=None, sort_order=None, \
                            until_id=None, user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id',
                   'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'])
                for tweet in res.data:
                    f.write(json.dumps(tweet.data, sort_keys=True)+'\n')
                next_token = res.meta.get('next_token', None)
                if not next_token:
                    done=True

            except TweepyException as e:
                print(e)
                errors.append(query)
                time.sleep(60)
                done=True

queries:  76%|███████▌  | 280/369 [8:19:38<36:46, 24.79s/it]    

503 Service Unavailable
Service Unavailable


queries:  98%|█████████▊| 361/369 [9:31:03<27:30, 206.37s/it]

400 Bad Request
There were errors processing your request: no viable alternative at input '<EOF>' (at position 889)


queries:  98%|█████████▊| 362/369 [9:32:03<18:57, 162.53s/it]

...

queries:  99%|█████████▉| 367/369 [9:40:09<03:32, 106.14s/it]

503 Service Unavailable
Service Unavailable


queries: 100%|█████████▉| 368/369 [10:03:09<08:08, 488.29s/it]

503 Service Unavailable
Service Unavailable


queries: 100%|██████████| 369/369 [10:07:26<00:00, 98.77s/it] 


In [105]:
remaining = errors[:]

In [106]:
out_path = os.path.join(DATA_PATH, f'italychurchtoo_nolang_convo.jsonl')
errors = list()
with open(out_path, 'a+') as f:
    for query in tqdm(remaining, 'queries'):
        done = False
        next_token = None
        while not done:
            try:
                res = client.search_all_tweets(query=query, next_token=next_token, max_results=500, start_time=start_date, end_time=None,\
                                               place_fields = ['id','full_name', 'country', 'geo', 'name', 'place_type'],
                                             tweet_fields = ['id', 'created_at', 'geo', 'public_metrics', 'text', 'edit_history_tweet_ids', 'attachments', 'author_id',  'conversation_id', 'entities', 'in_reply_to_user_id', 'lang', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'withheld'], # missing edit controls and  other fields that require user auth, 'context_annotations' requires <100 results
                            media_fields=['url', 'alt_text','public_metrics', 'type', 'media_key', 'duration_ms'],  \
                            poll_fields=['id', 'options', 'duration_minutes', 'end_datetime', 'voting_status'],
                                               since_id=None, sort_order=None, \
                            until_id=None, user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id',
                   'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'])
                for tweet in res.data:
                    f.write(json.dumps(tweet.data, sort_keys=True)+'\n')
                next_token = res.meta.get('next_token', None)
                if not next_token:
                    done=True

            except TweepyException as e:
                print(e)
                errors.append(query)
                time.sleep(60)
                done=True

queries:   0%|          | 0/4 [00:00<?, ?it/s]

400 Bad Request
There were errors processing your request: no viable alternative at input '<EOF>' (at position 889)


queries:  25%|██▌       | 1/4 [01:00<03:00, 60.23s/it]Rate limit exceeded. Sleeping for 97 seconds.
Rate limit exceeded. Sleeping for 304 seconds.
Rate limit exceeded. Sleeping for 213 seconds.
Rate limit exceeded. Sleeping for 40 seconds.
queries:  25%|██▌       | 1/4 [1:03:43<3:11:10, 3823.51s/it]


KeyboardInterrupt: 

In [104]:
len(errors)

4

In [101]:
errors[1]

'conversation_id:1415504184804261888 OR conversation_id:1030747586595106816 OR conversation_id:663248916092203010 OR conversation_id:635186714378526722 OR conversation_id:1609202384177070080 OR conversation_id:958688034572169218 OR conversation_id:1494051761694326787 OR conversation_id:1593585507564339203 OR conversation_id:488701944698253313 OR conversation_id:1142062643756318720 OR conversation_id:483511118733012992 OR conversation_id:1594143281050537986 OR conversation_id:1614608414440669184 OR conversation_id:983530482875936768 OR conversation_id:1609595624306970626 OR conversation_id:731893570324578304 OR conversation_id:1021773926337273857 OR conversation_id:978334518305320960 OR conversation_id:1144312015957180418 OR conversation_id:468739499061968896 OR conversation_id:1030461370645585920 OR conversation_id:1042014027994554368 OR conversation_id:1425916925863993356 OR conversation_id:849595040091574272 OR conversation_id:586431287776296960 OR conversation_id:1043584146608726016